## 🤗 Finetune **Longformer Encoder-Decoder (LED)** for Abstract Generation 🤗


---
This notebook is based on the training script provided with the [LED](https://huggingface.co/transformers/model_doc/led.html) model from the [Huggingface Transformers](https://huggingface.co/transformers/) library. The original script can be found [here](https://colab.research.google.com/drive/12LjJazBl7Gam0XBPy_y0CTOJZeZ34c2v?usp=sharing#scrollTo=6GRz0rksYb3h)


---
The *Longformer Encoder-Decoder (LED)* was recently added as an extension to [Longformer: The Long-Document Transformer](https://arxiv.org/abs/2004.05150) by Iz Beltagy, Matthew E. Peters, Arman Cohan.

In this notebook we will finetune *LED* for Summarization on [Pubmed](https://huggingface.co/datasets/viewer/?dataset=scientific_papers). *Pubmed* is a long-range summarization dataset, which makes it a good candidate for LED. LED will be finetuned up to an input length of 8K tokens on a single GPU.

We will leverage 🤗`Seq2SeqTrainer`, gradient checkpointing and as usual 🤗`datasets`.

Training this model takes a decently powerful GPU. The original notebook recommends a GPU with at least 15GB of VRAM. Fortunately, we have access to cloud computing resources, so we are able to do run experiments with thos model.

In [1]:
!nvidia-smi

Fri Nov 11 18:41:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    49W / 400W |      0MiB / 40960MiB |     27%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Install all of the packages needed for this project. We need to use the `-f https://download.pytorch.org/whl/torch_stable.html` flag to install the correct version of PyTorch for the GPU we are using.

In [2]:
!pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html
!export TOKENIZERS_PARALLELISM=false

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


## Dataset

Let's start by loading and preprocessing the dataset. NOTE: we will have to change this slightly when we switch to predicting the introduction instead of the abstract.

In [3]:
from datasets import load_dataset, load_metric

In [4]:
def lists_to_single_str(dataset):
    dataset['section_titles'] = '\n'.join(dataset['section_titles'])
    dataset['section_texts'] = '\n'.join(dataset['section_texts'])

    return dataset

# load the dataset from cnn_papers.json and nlp_papers.json (TODO: include ml_papers.json)
dataset = load_dataset('json', data_files=['cnn_papers.json', 'nlp_papers.json'], split='train')
dataset = dataset.map(lists_to_single_str) # convert the paper sections into a format that can be processed by the model

Using custom data configuration default-77495a8d85b716fa
Found cached dataset json (/home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-ee2ea3f94da7c697.arrow


Right now, the dataset should be split with an 80/20 train/test split. We may change this later to a train/val/test split.

In [5]:
seed = 42 # set the seed for reproducibility (and so that we can get the same test dataset when evaluating the model)

dataset = dataset.train_test_split(test_size=0.2, seed=seed)
print(len(dataset['train']))
print(len(dataset['test']))

Loading cached split indices for dataset at /home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-88af51e1849c8908.arrow and /home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-5b972d60d78c0067.arrow


78
20


Let's take a quick look at one of the papers

In [6]:
import random

paper = dataset['train'][random.randint(0, len(dataset['train']))]
print(paper['title'], '\n')
print(paper['section_titles'], '\n')
print(paper['abstract'])

Quantum Optical Convolutional Neural Network: A Novel Image Recognition Framework for Quantum Computing 

Introduction
Methods
Results
Conclusion 

Large machine learning models based on Convolutional Neural Networks (CNNs) with rapidly increasing numbers of parameters are being deployed in a wide array of computer vision tasks. However, the insatiable demand for computing resources required to train these models is fast outpacing the advancement of classical computing hardware, and new frameworks including Optical Neural Networks (ONNs) and quantum computing are being explored as future alternatives. In this work, we report a novel quantum computing based deep learning model, the Quantum Optical Convolutional Neural Network (QOCNN), to alleviate the computational bottleneck in future computer vision applications. Using the MNIST dataset, we have benchmarked this new architecture against a traditional CNN model based on the seminal LeNet architecture. We have also compared the performa

## Tokenizing

Now, we tokenize it using a Autotokenizer from HuggingFace.

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("debbiesoon/longformer_summarise")

Note that for the sake of this notebook, we finetune the "smaller" LED checkpoint ["allenai/led-base-16384"](https://huggingface.co/allenai/led-base-16384). Better performance can however be attained by finetuning ["allenai/led-large-16384"](https://huggingface.co/allenai/led-large-16384) at the cost of a higher required GPU RAM.

In [8]:
max_input_length =16384
max_output_length = 512
batch_size = 2

Now, let's write down the input data processing function that will be used to map each data sample to the correct model format.
As explained earlier `article` represents here our input data and `abstract` is the target data. The datasamples are thus tokenized up to the respective maximum lengths of 8192 and 512.

In addition to the usual `attention_mask`, LED can make use of an additional `global_attention_mask` defining which input tokens are attended globally and which are attended only locally, just as it's the case of [Longformer](https://huggingface.co/transformers/model_doc/longformer.html). For more information on Longformer's self-attention, please take a look at the corresponding [docs](https://huggingface.co/transformers/model_doc/longformer.html#longformer-self-attention). For summarization, we follow recommendations of the [paper](https://arxiv.org/abs/2004.05150) and use global attention only for the very first token. Finally, we make sure that no loss is computed on padded tokens by setting their index to `-100`.

In [9]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["section_texts"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

Now that we have a function that tokenizes the data, we can apply it to our dataset.

In [10]:
dataset = dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["title", "section_texts", "abstract", "section_titles"], # remove the columns that we don't need anymore since we've already tokenized them
)

Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-9fa5fe3bc8791cf0.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

Finally, the datasets should be converted into the PyTorch format as follows.

In [11]:
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

## Model

Alright, we're almost ready to start training. Let's load the model via the `AutoModelForSeq2SeqLM` class.

In [12]:
from transformers import AutoModelForSeq2SeqLM

We've decided to stick to the smaller model `"allenai/led-base-16384"` for the sake of this notebook. In addition, we directly enable gradient checkpointing and disable the caching mechanism to save memory.

In [13]:
led = AutoModelForSeq2SeqLM.from_pretrained("debbiesoon/longformer_summarise", use_cache=False)
led.gradient_checkpointing_enable()

During training, we want to evaluate the model on Rouge, the most common metric used in summarization, to make sure the model is indeed improving during training. For this, we set fitting generation parameters. We'll use beam search with a small beam of just 2 to save memory. Also, we force the model to generate at least 100 tokens, but no more than 512. In addition, some other generation parameters are set that have been found helpful for generation. For more information on those parameters, please take a look at the [docs](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate).

In [14]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = max_output_length
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

Next, we also have to define the function the will compute the `"rouge"` score during evalution.

Let's load the `"rouge"` metric from 🤗datasets and define the `compute_metrics(...)` function.

In [15]:
rouge = load_metric("rouge")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


The compute metrics function expects the generation output, called `pred.predictions` as well as the gold label, called `pred.label_ids`.

Those tokens are decoded and consequently, the rouge score can be computed.

In [16]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

## Training

Now, we're ready to start training. Let's import the `Seq2SeqTrainer` and `Seq2SeqTrainingArguments`.

In [17]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In contrast to the usual `Trainer`, the `Seq2SeqTrainer` makes it possible to use the `generate()` function during evaluation. This should be enabled with `predict_with_generate=True`. Because our GPU RAM is limited, we make use of gradient accumulation by setting `gradient_accumulation_steps=4` to have an effective `batch_size` of 2 * 4 = 8.

Other training arguments can be read upon in the [docs](https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.TrainingArguments).

In [18]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./",
    gradient_accumulation_steps=4,
    num_train_epochs=5, # since our dataset is so small, we may want to train for more epochs
)

The training arguments, along with the model, tokenizer, datasets and the `compute_metrics` function can then be passed to the `Seq2SeqTrainer`

In [19]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

Using cuda_amp half precision backend


Now we can start training.

In [20]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 78
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 45
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
0,2.972400,2.769551,0.145600,0.111500,0.123800
1,2.533000,2.746770,0.157100,0.122600,0.134300
2,2.289600,2.724972,0.159100,0.144600,0.149000
3,2.123300,2.741104,0.159600,0.153200,0.153200
4,1.911800,2.735474,0.170700,0.138500,0.149300


***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=45, training_loss=2.366032536824544, metrics={'train_runtime': 982.6173, 'train_samples_per_second': 0.397, 'train_steps_per_second': 0.046, 'total_flos': 4147514626277376.0, 'train_loss': 2.366032536824544, 'epoch': 4.92})

## Evaluation

Lets take a look at how well the original model compares to the fine-tuned model.

First, lets load the test dataset.

In [21]:
from datasets import load_dataset

def lists_to_single_str(dataset):
    dataset['section_titles'] = '\n'.join(dataset['section_titles'])
    dataset['section_texts'] = '\n'.join(dataset['section_texts'])

    return dataset

# load the dataset from cnn_papers.json and nlp_papers.json (TODO: include ml_papers.json)
dataset = load_dataset('json', data_files=['cnn_papers.json', 'nlp_papers.json'], split='train')
dataset = dataset.map(lists_to_single_str) # convert the paper sections into a format that can be processed by the model
seed = 42 # set the seed for reproducibility

dataset = dataset.train_test_split(test_size=0.2, seed=seed)

And we can define a function that will let us evaluate the model on the test dataset.

In [22]:
import torch
import copy
from datasets import load_metric

def evaluate_model(model, tokenizer):
    model.eval()

    def generate_answer(batch):
      inputs_dict = tokenizer(batch["section_texts"], padding="max_length", max_length=max_input_length, return_tensors="pt", truncation=True)
      input_ids = inputs_dict.input_ids.to("cuda")
      attention_mask = inputs_dict.attention_mask.to("cuda")
      global_attention_mask = torch.zeros_like(attention_mask)
      # put global attention on <s> token
      global_attention_mask[:, 0] = 1

      predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
      batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

      return batch

    result = dataset['test'].map(generate_answer, batched=True, batch_size=4)

    # load rouge
    rouge = load_metric("rouge")

    print("Rouge Results:", rouge.compute(predictions=result["predicted_abstract"], references=result["abstract"], rouge_types=["rouge2"])["rouge2"].mid)

    return result

Here are the results from the original LED-Base-16384 model.

In [23]:
from transformers import AutoTokenizer

original_tokenizer = AutoTokenizer.from_pretrained("debbiesoon/longformer_summarise")
original_model = AutoModelForSeq2SeqLM.from_pretrained("debbiesoon/longformer_summarise").to('cuda')

# set generate hyperparameters to be same as LED
original_model.config.num_beams = 2
original_model.config.max_length = max_output_length
original_model.config.min_length = 100
original_model.config.length_penalty = 2.0
original_model.config.early_stopping = True
original_model.config.no_repeat_ngram_size = 3

untrained_result = evaluate_model(original_model, original_tokenizer)

loading file vocab.json from cache at /home/jupyter/.cache/huggingface/hub/models--debbiesoon--longformer_summarise/snapshots/6ece36677a3126502073da31a9f90299a1766d32/vocab.json
loading file merges.txt from cache at /home/jupyter/.cache/huggingface/hub/models--debbiesoon--longformer_summarise/snapshots/6ece36677a3126502073da31a9f90299a1766d32/merges.txt
loading file tokenizer.json from cache at /home/jupyter/.cache/huggingface/hub/models--debbiesoon--longformer_summarise/snapshots/6ece36677a3126502073da31a9f90299a1766d32/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jupyter/.cache/huggingface/hub/models--debbiesoon--longformer_summarise/snapshots/6ece36677a3126502073da31a9f90299a1766d32/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jupyter/.cache/huggingface/hub/models--debbiesoon--longformer_summarise/snapshots/6ece36677a3126502073da31a9f90299a1766d32/tokenizer_config.json
load

  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Rouge Results: Score(precision=0.11683710243487759, recall=0.10326144519886835, fmeasure=0.10017863269463509)


And here are the results from the fine-tuned model.

In [24]:
result = evaluate_model(led, tokenizer)

  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Rouge Results: Score(precision=0.17114033406383417, recall=0.13870173744324893, fmeasure=0.15014662772052517)


Let's compare an example abstract from the test dataset to the abstracts predicted by the fine-tuned and untrained models.

Here is the original abstract:

In [25]:
index = random.randint(0, len(result))
print(dataset['test'][index]['abstract'])

'Incorporating prior knowledge into recurrent neural network (RNN) is of great importance for many natural language processing tasks. However, most of the prior knowledge is in the form of structured knowledge and is difficult to be exploited in the existing RNN framework. By extracting the logic rules from the structured knowledge and embedding the extracted logic rule into the RNN, this paper proposes an effective framework to incorporate the prior information in the RNN models. First, we demonstrate that commonly used prior knowledge could be decomposed into a set of logic rules, including the knowledge graph, social graph, and syntactic dependence. Second, we present a technique to embed a set of logic rules into the RNN by the way of feedback masks. Finally, we apply the proposed approach to the sentiment classification and named entity recognition task. The extensive experimental results verify the effectiveness of the embedding approach. The encouraging results suggest that the 

Here is the abstract predicted by the untrained model:

In [26]:
print(untrained_result[index]['predicted_abstract']

' background : we propose a method to embed the logic rules into recurrent neural networks (RNNs). This method is based on the probabilistic graphical model and neural network (NN) framework. \n the underlying logic rules of the recurrent neural network are described in this paper. The logic rules are derived from a set of conditional probability tables associated with the neural networks. The data are stored in a single data set and are stored on the same data set. The underlying logic rule is used to represent the information of the neural network. The parameters of the logic rule are set to 0.01 and 1.0, respectively. The values of the data are expressed in a matrix of 0 and 1*d, respectively, and the value of the conditional probability table is set to 1 for the input and 0.88 for the output. '

And lastly, here is the abstract predicted by the fine-tuned model:

In [27]:
result[index]['predicted_abstract']

'Recurrent neural network (RNN) works well as sequence models because they are able to represent natural language sequences, and have been successfully applied to a variety of NLP tasks, including speech recognition [1], machine translation [2], sentiment classification [3], and named entity recognition. In this paper, we propose a general and concise way to embed logic rules into recurrent neural networks. We propose a method to embed the logic rules in a recurrent neural network, which provides a unified presentation of the various prior knowledge in terms of Acc and sentiment classification. The embedding of logic rules is a general, concise approach to incorporate prior knowledge into the RNN framework. The experimental results and discussions are presented in this paper.'